In [1]:
import sys
sys.path.append('..')
from MPRA_predict.utils import *
from MPRA_predict.metrics import *

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.neural_network import MLPRegressor, MLPClassifier

In [2]:
def find_indice(info_df, cell_types=[], assays=[], IDs=[]):
    # 初始化索引为全部行
    indice = list(range(len(info_df)))
    
    # 如果 cell_types 不为空，则筛选 cell_type
    if cell_types:
        indice = [i for i in indice if info_df['cell_type'][i] in cell_types]
        
    # 如果 assays 不为空，则筛选 assay
    if assays:
        indice = [i for i in indice if info_df['assay'][i] in assays]
        
    # 如果 IDs 不为空，则筛选 ID
    if IDs:
        indice = [i for i in indice if info_df['ID'][i] in IDs]
    
    return indice

In [3]:
# info_df = pd.read_csv('../pretrained_models/Sei/Sei_info.csv')
# print(info_df.shape)

# pretrained_pred = np.load('../pretrained_models/Sei/Sei_Siraj_pred_3.npy')
# print(pretrained_pred.shape)

# indice = find_indice(info_df, IDs=['ENCODE', 'Roadmap'])
# info_df.iloc[indice].to_csv('../pretrained_models/Sei/Sei_info_2002.csv', index=False)
# np.save('../pretrained_models/Sei/Sei_Siraj_pred_2002.npy', pretrained_pred[:, indice])

In [5]:
info_df = pd.read_csv('data/Sei_info.csv')
info_df

,cell_type,assay,ID,part
0,HeLa_Epithelium_Cervix,BTAF1,ID:1,0
1,H9_Embryonic_Stem_Cell_Embryo,EOMES,ID:1006,0
2,H9_Embryonic_Stem_Cell_Embryo,EP300,ID:1007,0
3,H9_Embryonic_Stem_Cell_Embryo,FOXH1,ID:1010,0
4,H9_Embryonic_Stem_Cell_Embryo,FOXH1,ID:1011,0
...,...,...,...,...
21902,WERI-Rb-1,CTCF,ENCODE,0
21903,WERI-Rb-1,DNase,ENCODE,0
21904,WI-38,CTCF,ENCODE,0
21905,WI-38 (4OHTAM_20nM_72hr),DNase,ENCODE,0


In [6]:
pretrained_pred = np.load('../pretrained_models/Sei/Sei_Siraj_pred.npy')
print(pretrained_pred.shape)

main_df = pd.read_csv('/home/hxcai/cell_type_specific_CRE/data/SirajMPRA/SirajMPRA_total.csv')
print(main_df.shape)

FileNotFoundError: [Errno 2] No such file or directory: '../pretrained_models/Sei/Sei_Siraj_pred.npy'

In [16]:
# pretrained_pred = np.load('../pretrained_models/Sei/Sei_Agarwal_pred.npy')
# print(pretrained_pred.shape)

# main_df = pd.read_csv('/home/hxcai/cell_type_specific_CRE/data/AgarwalMPRA/Agarwal.csv')
# print(main_df.shape)

In [17]:
# pretrained_pred = np.load('../pretrained_models/Sei/Sei_Siraj_pred.npy')
# print(pretrained_pred.shape)

# pred = np.load('../pretrained_models/Sei/Sei_Gosai_designed_pred.npy')
# pretrained_pred = np.concatenate([pretrained_pred, pred])
# print(pretrained_pred.shape)


# main_df = pd.read_csv('/home/hxcai/cell_type_specific_CRE/data/SirajMPRA/SirajMPRA_total.csv')
# print(main_df.shape)

# df = pd.read_csv('/home/hxcai/cell_type_specific_CRE/data/GosaiMPRA/GosaiMPRA_designed.csv')
# df['chr'] = 'chr0'
# main_df = pd.concat([main_df, df]).reset_index(drop=True)
# print(main_df.shape)

In [19]:
# # 定义细胞所有可能的名字
aliases_dict = {
    'HepG2': [
        r"hepg2",
        r"hepatocellular[_\s-]*carcinoma",
        r"hep[_\s-]*g[_\s-]*2",
        ],
    'K562': [
        r"K562",
    ],
    'SKNSH': [
        r"neuroblastoma",
        r"SKNSH",
        r"SK-N-SH",
    ],
    'A549': [
        r"A549",
    ],
    'HCT116': [
        r"HCT116",
        r"colon",
    ],
    'WTC11': [
        r"WTC11",
        r"iPS",
        r"GM23338",
    ],
}


assays = ['DNase', 'H3K4me1', 'H3K4me3', 'H3K9me3', 'H3K27me3', 'H3K27ac', 'H3K36me3', 'CTCF']
cell_types = ['HepG2', 'K562', 'SKNSH', 'A549', 'HCT116']
info_df['pearson_r'] = np.nan

for cell_type in cell_types:
    aliases = aliases_dict[cell_type]
    pattern = re.compile("|".join(aliases), re.IGNORECASE) # 不区分大小写
    matched_indice = [i for i in range(len(info_df))
                      if (pattern.search(info_df['cell_type'][i])) and (info_df['assay'][i] in assays)]
    
    for i in matched_indice:
        x = logit(pretrained_pred[:, i])
        y = main_df[cell_type]
        r = pearson(x, y)
        info_df.loc[i, 'pearson_r'] = r

    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 10000)
    print(info_df.iloc[matched_indice].sort_values(by='cell_type'))
    pd.set_option('display.max_rows', 10)
    cell_type_names = info_df.iloc[matched_indice]['cell_type']
    counter = Counter(cell_type_names)
    for name in counter:
        print(f'{name}: {counter[name]}')

                            cell_type     assay        ID  part  pearson_r
20977                           HepG2     DNase    ENCODE     0   0.393468
20976                           HepG2      CTCF    ENCODE     3   0.380165
20975                           HepG2      CTCF    ENCODE     2   0.366790
20974                           HepG2      CTCF    ENCODE     1   0.346193
20973                           HepG2      CTCF    ENCODE     0   0.391314
144            HepG2_Epithelium_Liver      CTCF   ID:1380     0   0.300316
4850           HepG2_Epithelium_Liver      CTCF  ID:45303     0   0.345006
4852           HepG2_Epithelium_Liver   H3K4me3  ID:45306     0   0.440103
5311           HepG2_Epithelium_Liver      CTCF  ID:45794     0   0.321988
5312           HepG2_Epithelium_Liver      CTCF  ID:45795     0   0.370436
5325           HepG2_Epithelium_Liver      CTCF  ID:45808     0   0.291261
5765           HepG2_Epithelium_Liver      CTCF  ID:46250     0   0.324868
7480           HepG2_Epit

In [22]:
cell_types_names_dict = {
    'HepG2': ['HepG2_Epithelium_Liver'],
    'K562' : ['K562_Erythroblast_Bone_Marrow'],
    'SKNSH': ['SK-N-SH_Neuroblastoma_cell_Brain'],
    'A549' : ['A549_Epithelium_Lung'],
    'HCT116':['HCT-116_HCT116', 'HCT-116_Colon'], # HCT-116_HCT116效果好，HCT-116_Colon通道多
    'WTC11': ['iPSC'],
}


warnings.filterwarnings("ignore", category=RuntimeWarning)

for cell_type in cell_types:
    for assay in assays:
        cell_names = cell_types_names_dict[cell_type]
        matched_indice = find_indice(info_df, cell_names, [assay])
        value = logit(pretrained_pred[:, matched_indice]).mean(axis=1) #先logit再mean
        main_df[f'{cell_type}_{assay}_pred_logit'] = value

        y = main_df[cell_type]
        r = pearson(value, y)
        print(f'{cell_type} {assay}, found {len(matched_indice)} records, pearson r = {r:.3f}')

main_df = main_df.copy()

HepG2 DNase, found 7 records, pearson r = 0.444
HepG2 H3K4me1, found 3 records, pearson r = 0.351
HepG2 H3K4me3, found 7 records, pearson r = 0.466
HepG2 H3K9me3, found 1 records, pearson r = 0.015
HepG2 H3K27me3, found 1 records, pearson r = 0.255
HepG2 H3K27ac, found 8 records, pearson r = 0.426
HepG2 H3K36me3, found 2 records, pearson r = 0.324
HepG2 CTCF, found 13 records, pearson r = 0.341
K562 DNase, found 56 records, pearson r = 0.499
K562 H3K4me1, found 9 records, pearson r = 0.391
K562 H3K4me3, found 90 records, pearson r = 0.450
K562 H3K9me3, found 20 records, pearson r = 0.285
K562 H3K27me3, found 33 records, pearson r = 0.189
K562 H3K27ac, found 50 records, pearson r = 0.442
K562 H3K36me3, found 4 records, pearson r = 0.345
K562 CTCF, found 22 records, pearson r = 0.357
SKNSH DNase, found 0 records, pearson r = nan
SKNSH H3K4me1, found 0 records, pearson r = nan
SKNSH H3K4me3, found 3 records, pearson r = 0.484
SKNSH H3K9me3, found 0 records, pearson r = nan
SKNSH H3K27me3,

In [23]:
main_df.to_csv('data/Sei_Siraj_features.csv', index=False)

In [3]:
main_df = pd.read_csv('data/Sei_Siraj_features.csv')
main_df

,seq,A549,HepG2,K562,SKNSH,HCT116,is_ref,chr,pos,ref,alt,HepG2_DNase_pred_logit,HepG2_H3K4me1_pred_logit,HepG2_H3K4me3_pred_logit,HepG2_H3K9me3_pred_logit,HepG2_H3K27me3_pred_logit,HepG2_H3K27ac_pred_logit,HepG2_H3K36me3_pred_logit,HepG2_CTCF_pred_logit,K562_DNase_pred_logit,K562_H3K4me1_pred_logit,K562_H3K4me3_pred_logit,K562_H3K9me3_pred_logit,K562_H3K27me3_pred_logit,K562_H3K27ac_pred_logit,K562_H3K36me3_pred_logit,K562_CTCF_pred_logit,SKNSH_DNase_pred_logit,SKNSH_H3K4me1_pred_logit,SKNSH_H3K4me3_pred_logit,SKNSH_H3K9me3_pred_logit,SKNSH_H3K27me3_pred_logit,SKNSH_H3K27ac_pred_logit,SKNSH_H3K36me3_pred_logit,SKNSH_CTCF_pred_logit,A549_DNase_pred_logit,A549_H3K4me1_pred_logit,A549_H3K4me3_pred_logit,A549_H3K9me3_pred_logit,A549_H3K27me3_pred_logit,A549_H3K27ac_pred_logit,A549_H3K36me3_pred_logit,A549_CTCF_pred_logit,HCT116_DNase_pred_logit,HCT116_H3K4me1_pred_logit,HCT116_H3K4me3_pred_logit,HCT116_H3K9me3_pred_logit,HCT116_H3K27me3_pred_logit,HCT116_H3K27ac_pred_logit,HCT116_H3K36me3_pred_logit,HCT116_CTCF_pred_logit
0,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,NaN,0.667516,-0.704147,0.538641,0.159069,True,chr1,14677,G,A,-5.245570,-6.484603,-5.046573,-3.432081,-4.963858,-7.134384,-8.048167,-5.811829,-3.848695,-5.481934,-5.635548,-3.213235,-6.120426,-6.389701,-8.903847,-5.937818,NaN,NaN,-5.204737,NaN,NaN,-6.789738,NaN,-6.166427,-3.909284,-6.719756,-4.348619,-6.747242,-4.744374,-6.072631,-8.451625,-5.178783,-4.183375,-5.234190,-4.748006,-5.354374,-5.090679,-5.958753,-7.642133,-5.942789
1,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,NaN,0.415984,0.554099,0.441904,-0.288019,False,chr1,14677,G,A,-5.284457,-6.443999,-5.064491,-3.456125,-4.954262,-7.130351,-8.050102,-5.751132,-3.889245,-5.505225,-5.662835,-3.259038,-6.133786,-6.381230,-8.945845,-5.900684,NaN,NaN,-5.308731,NaN,NaN,-6.830903,NaN,-6.152922,-3.935934,-6.752880,-4.430842,-6.787907,-4.786582,-6.121103,-8.504293,-5.139558,-4.202863,-5.317146,-4.801554,-5.399591,-5.143095,-5.995626,-7.684950,-5.899889
2,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,NaN,1.269937,-0.391222,0.752052,1.100437,True,chr1,63697,T,C,-5.832485,-6.215046,-6.275543,-4.486951,-6.938931,-8.309289,-10.355912,-7.573901,-5.905354,-7.871817,-8.285551,-6.839465,-7.052541,-8.233128,-10.560387,-8.483019,NaN,NaN,-6.299528,NaN,NaN,-6.223368,NaN,-7.684988,-3.272244,-6.935732,-5.964360,-8.286272,-5.773897,-7.547378,-9.174150,-7.003038,-4.215008,-5.436987,-6.929256,-5.848139,-5.366335,-7.071730,-9.561171,-7.614042
3,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,NaN,1.151995,-0.659023,0.204721,1.050619,False,chr1,63697,T,C,-5.843955,-6.331291,-6.612215,-4.268522,-7.028302,-8.509892,-10.369975,-7.567623,-6.080841,-8.033148,-8.534241,-6.809958,-7.087788,-8.399063,-10.582304,-8.564973,NaN,NaN,-6.753442,NaN,NaN,-6.580954,NaN,-7.882953,-3.276257,-7.151476,-6.369055,-8.265885,-5.863884,-7.888105,-9.129220,-7.052451,-4.185768,-5.522015,-7.128210,-5.841283,-5.615144,-7.231531,-9.487424,-7.640062
4,AATAGACTCTAGTACTCTGTATTATGCAAAATTTGTCTATGTTACA...,NaN,-0.211076,-0.007174,0.499376,0.257572,True,chr1,64764,C,T,-10.333365,-8.949069,-9.762632,-4.353646,-7.956873,-10.991928,-11.289205,-10.315630,-7.917572,-9.117440,-10.260711,-7.934627,-7.752861,-9.121041,-12.105309,-9.972706,NaN,NaN,-8.882309,NaN,NaN,-9.244476,NaN,-10.126541,-8.595281,-9.953346,-9.448072,-9.112960,-6.309947,-10.267394,-10.511040,-9.144752,-7.840627,-8.620074,-9.203153,-5.563872,-5.998085,-9.125849,-11.204397,-9.197061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585600,ATCATTTACTGGTGACTGCCCTGCTAGGGTGTCAGCTGTCAGATCC...,NaN,0.615976,1.927090,0.798263,1.035440,False,chrX,155233098,T,C,-3.483061,-4.722695,-4.618957,-3.777600,-6.374849,-5.802739,-8.910620,-5.265605,-3.845181,-5.772332,-6.744490,-4.107216,-6.837776,-6.754680,-9.625647,-6.639157,NaN,NaN,-6.714044,NaN,NaN,-7.724927,NaN,-6.4

In [8]:
# main_df = main_df.drop_duplicates(subset=['seq'])
# main_df

,seq,A549,HepG2,K562,SKNSH,HCT116,is_ref,chr,pos,ref,alt,HepG2_DNase_pred_logit,HepG2_H3K4me1_pred_logit,HepG2_H3K4me3_pred_logit,HepG2_H3K9me3_pred_logit,HepG2_H3K27me3_pred_logit,HepG2_H3K27ac_pred_logit,HepG2_H3K36me3_pred_logit,HepG2_CTCF_pred_logit,K562_DNase_pred_logit,K562_H3K4me1_pred_logit,K562_H3K4me3_pred_logit,K562_H3K9me3_pred_logit,K562_H3K27me3_pred_logit,K562_H3K27ac_pred_logit,K562_H3K36me3_pred_logit,K562_CTCF_pred_logit,SKNSH_DNase_pred_logit,SKNSH_H3K4me1_pred_logit,SKNSH_H3K4me3_pred_logit,SKNSH_H3K9me3_pred_logit,SKNSH_H3K27me3_pred_logit,SKNSH_H3K27ac_pred_logit,SKNSH_H3K36me3_pred_logit,SKNSH_CTCF_pred_logit,A549_DNase_pred_logit,A549_H3K4me1_pred_logit,A549_H3K4me3_pred_logit,A549_H3K9me3_pred_logit,A549_H3K27me3_pred_logit,A549_H3K27ac_pred_logit,A549_H3K36me3_pred_logit,A549_CTCF_pred_logit,HCT116_DNase_pred_logit,HCT116_H3K4me1_pred_logit,HCT116_H3K4me3_pred_logit,HCT116_H3K9me3_pred_logit,HCT116_H3K27me3_pred_logit,HCT116_H3K27ac_pred_logit,HCT116_H3K36me3_pred_logit,HCT116_CTCF_pred_logit
0,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,NaN,0.667516,-0.704147,0.538641,0.159069,True,chr1,14677,G,A,-5.245570,-6.484603,-5.046573,-3.432081,-4.963858,-7.134384,-8.048167,-5.811829,-3.848695,-5.481934,-5.635548,-3.213235,-6.120426,-6.389701,-8.903847,-5.937818,NaN,NaN,-5.204737,NaN,NaN,-6.789738,NaN,-6.166427,-3.909284,-6.719756,-4.348619,-6.747242,-4.744374,-6.072631,-8.451625,-5.178783,-4.183375,-5.234190,-4.748006,-5.354374,-5.090679,-5.958753,-7.642133,-5.942789
1,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,NaN,0.415984,0.554099,0.441904,-0.288019,False,chr1,14677,G,A,-5.284457,-6.443999,-5.064491,-3.456125,-4.954262,-7.130351,-8.050102,-5.751132,-3.889245,-5.505225,-5.662835,-3.259038,-6.133786,-6.381230,-8.945845,-5.900684,NaN,NaN,-5.308731,NaN,NaN,-6.830903,NaN,-6.152922,-3.935934,-6.752880,-4.430842,-6.787907,-4.786582,-6.121103,-8.504293,-5.139558,-4.202863,-5.317146,-4.801554,-5.399591,-5.143095,-5.995626,-7.684950,-5.899889
2,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,NaN,1.269937,-0.391222,0.752052,1.100437,True,chr1,63697,T,C,-5.832485,-6.215046,-6.275543,-4.486951,-6.938931,-8.309289,-10.355912,-7.573901,-5.905354,-7.871817,-8.285551,-6.839465,-7.052541,-8.233128,-10.560387,-8.483019,NaN,NaN,-6.299528,NaN,NaN,-6.223368,NaN,-7.684988,-3.272244,-6.935732,-5.964360,-8.286272,-5.773897,-7.547378,-9.174150,-7.003038,-4.215008,-5.436987,-6.929256,-5.848139,-5.366335,-7.071730,-9.561171,-7.614042
3,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,NaN,1.151995,-0.659023,0.204721,1.050619,False,chr1,63697,T,C,-5.843955,-6.331291,-6.612215,-4.268522,-7.028302,-8.509892,-10.369975,-7.567623,-6.080841,-8.033148,-8.534241,-6.809958,-7.087788,-8.399063,-10.582304,-8.564973,NaN,NaN,-6.753442,NaN,NaN,-6.580954,NaN,-7.882953,-3.276257,-7.151476,-6.369055,-8.265885,-5.863884,-7.888105,-9.129220,-7.052451,-4.185768,-5.522015,-7.128210,-5.841283,-5.615144,-7.231531,-9.487424,-7.640062
4,AATAGACTCTAGTACTCTGTATTATGCAAAATTTGTCTATGTTACA...,NaN,-0.211076,-0.007174,0.499376,0.257572,True,chr1,64764,C,T,-10.333365,-8.949069,-9.762632,-4.353646,-7.956873,-10.991928,-11.289205,-10.315630,-7.917572,-9.117440,-10.260711,-7.934627,-7.752861,-9.121041,-12.105309,-9.972706,NaN,NaN,-8.882309,NaN,NaN,-9.244476,NaN,-10.126541,-8.595281,-9.953346,-9.448072,-9.112960,-6.309947,-10.267394,-10.511040,-9.144752,-7.840627,-8.620074,-9.203153,-5.563872,-5.998085,-9.125849,-11.204397,-9.197061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585600,ATCATTTACTGGTGACTGCCCTGCTAGGGTGTCAGCTGTCAGATCC...,NaN,0.615976,1.927090,0.798263,1.035440,False,chrX,155233098,T,C,-3.483061,-4.722695,-4.618957,-3.777600,-6.374849,-5.802739,-8.910620,-5.265605,-3.845181,-5.772332,-6.744490,-4.107216,-6.837776,-6.754680,-9.625647,-6.639157,NaN,NaN,-6.714044,NaN,NaN,-7.724927,NaN,-6.4

In [4]:
result_df = pd.DataFrame()

# 定义细胞类型和对应的列前缀
cell_types = ['A549', 'HepG2', 'K562', 'SKNSH', 'HCT116']
assays = ['DNase', 'H3K4me1', 'H3K4me3', 'H3K9me3', 'H3K27me3', 'H3K27ac', 'H3K36me3', 'CTCF']

# 遍历每种细胞类型
for cell_type in cell_types:
    # 提取该细胞类型的相关列
    cols = ['seq', 'is_ref', 'chr', 'pos', 'ref', 'alt'] + [cell_type] + [f"{cell_type}_{assay}_pred_logit" for assay in assays]
    temp_df = main_df[cols].copy()
    # 重命名列，使其不包含细胞类型前缀
    temp_df.columns = ['seq', 'is_ref', 'chr', 'pos', 'ref', 'alt'] + ['exp'] + assays
    # 添加一个新的列用于标注细胞类型
    temp_df['cell_type'] = cell_type
    temp_df['index'] = temp_df.index
    # 将 temp_df 添加到 result_df 中
    result_df = pd.concat([result_df, temp_df], ignore_index=True)

# 显示结果 DataFrame
result_df

,seq,is_ref,chr,pos,ref,alt,exp,DNase,H3K4me1,H3K4me3,H3K9me3,H3K27me3,H3K27ac,H3K36me3,CTCF,cell_type,index
0,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,True,chr1,14677,G,A,NaN,-3.909284,-6.719756,-4.348619,-6.747242,-4.744374,-6.072631,-8.451625,-5.178783,A549,0
1,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,False,chr1,14677,G,A,NaN,-3.935934,-6.752880,-4.430842,-6.787907,-4.786582,-6.121103,-8.504293,-5.139558,A549,1
2,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,True,chr1,63697,T,C,NaN,-3.272244,-6.935732,-5.964360,-8.286272,-5.773897,-7.547378,-9.174150,-7.003038,A549,2
3,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,False,chr1,63697,T,C,NaN,-3.276257,-7.151476,-6.369055,-8.265885,-5.863884,-7.888105,-9.129220,-7.052451,A549,3
4,AATAGACTCTAGTACTCTGTATTATGCAAAATTTGTCTATGTTACA...,True,chr1,64764,C,T,NaN,-8.595281,-9.953346,-9.448072,-9.112960,-6.309947,-10.267394,-10.511040,-9.144752,A549,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2928020,ATCATTTACTGGTGACTGCCCTGCTAGGGTGTCAGCTGTCAGATCC...,False,chrX,155233098,T,C,1.035440,-3.711086,-4.699247,-5.555646,-6.362927,-6.117311,-6.364144,-8.306399,-6.310446,HCT116,585600
2928021,TGGTTACTGCAGGGGCAGGGTTTTGGCAGGAAATAAACATGCACGG...,True,chrX,155233794,A,G,1.495515,-2.356441,-4.035868,-5.232915,-6.058248,-5.297648,-5.920075,-8.679003,-6.512767,HCT116,585601
2928022,TGGTTACTGCAGGGGCAGGGTTTTGGCAGGAAATAAACATGCACGG...,False,chrX,155233794,A,G,0.794848,-3.112125,-4.560548,-5.147340,-5.923994,-5.302712,-6.142751,-8.524772,-6.767405,HCT116,585602
2928023,TGGTGAGTTGCTTCTGTGCACACACTTTTGTTTATGAGTGTGCATG...,True,chrX,155234730,T,C,0.444472,-5.317532,-6.502365,-8.150496,-5.235864,-5.065353,-8.504182,-8.218654,-8.253539,HCT116,585603


In [5]:
result_df = result_df.dropna().reset_index(drop=True)
result_df

,seq,is_ref,chr,pos,ref,alt,exp,DNase,H3K4me1,H3K4me3,H3K9me3,H3K27me3,H3K27ac,H3K36me3,CTCF,cell_type,index
0,CCCAGGTCCCCTCGCCGAGCCGCACCCGCTCTTTGCCACTGATCTC...,True,chr1,881627,G,A,0.174087,-5.069787,-6.677926,-4.772376,-7.496990,-5.175857,-6.617980,-8.960541,-6.203494,A549,143
1,CCCAGGTCCCCTCGCCGAGCCGCACCCGCTCTTTGCCACTGATCTC...,False,chr1,881627,G,A,-0.158530,-5.277000,-6.856542,-5.130377,-7.606262,-5.427920,-6.967863,-9.007917,-6.398984,A549,144
2,GGACAGATGGAGGTCACGGGAGGCCTGGGGGGCCCCTCCCACACCC...,True,chr1,884551,GAGAA,G,1.039451,-4.926292,-7.165932,-8.245449,-7.792621,-6.806868,-8.091847,-8.821148,-4.710222,A549,147
3,AGGAGTATGGACAGGACTTACAAGTTCTTACAAAGGGAAATAGAGC...,True,chr1,891059,C,T,1.404402,-3.559333,-6.580040,-5.233942,-7.777295,-5.029692,-6.400854,-8.375984,-6.433165,A549,158
4,AGGAGTATGGACAGGACTTACAAGTTCTTACAAAGGGAAATAGAGC...,False,chr1,891059,C,T,0.978974,-4.144285,-6.864729,-5.579468,-7.748142,-5.126643,-6.441888,-8.451361,-6.849373,A549,159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1765213,ATCATTTACTGGTGACTGCCCTGCTAGGGTGTCAGCTGTCAGATCC...,False,chrX,155233098,T,C,1.035440,-3.711086,-4.699247,-5.555646,-6.362927,-6.117311,-6.364144,-8.306399,-6.310446,HCT116,585600
1765214,TGGTTACTGCAGGGGCAGGGTTTTGGCAGGAAATAAACATGCACGG...,True,chrX,155233794,A,G,1.495515,-2.356441,-4.035868,-5.232915,-6.058248,-5.297648,-5.920075,-8.679003,-6.512767,HCT116,585601
1765215,TGGTTACTGCAGGGGCAGGGTTTTGGCAGGAAATAAACATGCACGG...,False,chrX,155233794,A,G,0.794848,-3.112125,-4.560548,-5.147340,-5.923994,-5.302712,-6.142751,-8.524772,-6.767405,HCT116,585602
1765216,TGGTGAGTTGCTTCTGTGCACACACTTTTGTTTATGAGTGTGCATG...,True,chrX,155234730,T,C,0.444472,-5.317532,-6.502365,-8.150496,-5.235864,-5.065353,-8.504182,-8.218654,-8.253539,HCT116,585603


In [7]:
result_df.to_csv('data/Sei_Siraj_features_concat.csv', index=False)